## Scaling your workflow with Dask

This notebook goes through some examples using Dask to scale some common workflows on tabular data.

Dask works either on a single machine (executing in parallel using threads) or on a cluster of many machines. The examples here will run on a cluster, just for fun, but Dask is also useful for working with larger-than-memory datasets on a single machine.

When using Dask on a Cluster the typical pattern is to

1. Create a Dask Cluster (using one of our many [deployment options](https://docs.dask.org/en/latest/setup.html) that talks to the resource manager)
2. Connect to the cluster with a local Client

In [ ]:
# This example uses Coiled, a for-profit company that will
# manage Dask deployments for you. You could also do it yourself
# and use one of
# * dask_ssh.SSHCluster()
# * dask_yarn.YarnCluster()
# * dask_jobqueue.PBSCluster()
# * dask_kubernetes.KubeCluster()
# * dask_gateway.GatewayCluster()
# * dask_cloudprovider.FargateCluster()
# * dask_cloudprovider.AzureMLCluster()
# * dask_saturn.SaturnCluster()
# * ...

Once we have a cluster (coiled, PBS, kubernetes, or otherwise), connect to it. After this, all Dask-backed operations will happen on the cluster.

In [ ]:
from distributed import Client

client = Client(cluster)
client

In [1]:
import pandas as pd

dtype={
    "payment_type": "UInt8",
    "VendorID": "UInt8",
    "passenger_count": "UInt8",
    "RatecodeID": "UInt8",
}

df = pd.read_csv(
    "s3://nyc-tlc/trip data/yellow_tripdata_2019-01.csv",
    dtype=dtype,
    parse_dates=["tpep_pickup_datetime", "tpep_dropoff_datetime"],
    nrows=1000
)
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1,2019-01-01 00:46:40,2019-01-01 00:53:20,1,1.50,1,N,151,239,1,7.0,0.5,0.5,1.65,0.0,0.3,9.95,NaN
1,1,2019-01-01 00:59:47,2019-01-01 01:18:59,1,2.60,1,N,239,246,1,14.0,0.5,0.5,1.00,0.0,0.3,16.30,NaN
2,2,2018-12-21 13:48:30,2018-12-21 13:52:40,3,0.00,1,N,236,236,1,4.5,0.5,0.5,0.00,0.0,0.3,5.80,NaN
3,2,2018-11-28 15:52:25,2018-11-28 15:55:45,5,0.00,1,N,193,193,2,3.5,0.5,0.5,0.00,0.0,0.3,7.55,NaN
4,2,2018-11-28 15:56:57,2018-11-28 15:58:33,5,0.00,2,N,193,193,2,52.0,0.0,0.5,0.00,0.0,0.3,55.55,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2,2018-12-31 23:56:51,2019-01-01 00:07:25,4,3.47,1,N,161,151,1,12.0,0.5,0.5,2.66,0.0,0.3,15.96,NaN
996,2,2019-01-01 00:12:12,2019-01-01 00:16:58,4,0.55,1,N,238,151,1,5.0,0.5,0.5,1.89,0.0,0.3,8.19,NaN
997,2,2019-01-01 00:22:03,2019-01-01 00:34:30,5,2.30,1,N,238,141,2,11.0,0.5,0.5,0.00,0.0,0.3,12.30,NaN
998,2,2019-01-01 00:35:42,2019-01-01 00:56:04,5,5.89,1,N,141,144,1,20.5,0.5,0.5,4.00,0.0,0.3,25.80,NaN


In [ ]:
counts = df.passenger_count.value_counts()
counts.sort_index().plot.bar(rot=0, width=1, color='k');

In [ ]:
df.groupby("passenger_count").tip_amount.mean()

In [ ]:
ddf.fare_amount.quantile([0.10, 0.25, 0.5, .75, 0.9]).compute()

Dask DataFrame mimics the pandas API. This means *many* of the
APIs you're familiar with will work with Dask. There are often
some dask-specific keywords as well, reflecting the fact that
parallel / distributed computing has its own set of concerns.

Dask's readers typically accept a list of URLs / files, or a globstring indicating a list of files to read in.

In [ ]:
import dask.dataframe as dd

ddf = dd.read_csv(
    "s3://nyc-tlc/trip data/yellow_tripdata_2019-*.csv",
    dtype=dtype,
    parse_dates=["tpep_pickup_datetime", "tpep_dropoff_datetime"],
    storage_options={"anon": True},
    blocksize="16 MiB",
)
ddf

A few things to note:

1. We reused the `dtype` and `parse_dates` options, just like before
2. We have additional dask-specific options like `blocksize`
3. The result returned almost instantly
4. The values in the table aren't shown, just the structure (column names and dtypes)

Dask's high-level collections like `dask.dataframe` are lazy. The just do enough work to propagate metadata for operations, like the type of the output, the column names and dtypes, etc.

In [ ]:
dask_counts = ddf.passenger_count.value_counts()
dask_counts

In [ ]:
ax = (
    dask_counts
    .compute()
    .sort_index()
    .plot.bar(rot=0, width=1, color='k')
);

Let's take a look at the profiler to see where time was spent.

In [ ]:
ddf = ddf.persist()

And now, let's re-run the computation.

In [ ]:
dask_counts = ddf.passenger_count.value_counts()
ax = (
    dask_counts
    .compute()
    .sort_index()
    .plot.bar(rot=0, width=1, color='k')
);

`dask.dataframe` supports many of the pandas APIs.

In [ ]:
ddf.groupby("passenger_count").tip_amount.mean().compute()

In [ ]:
ddf.fare_amount.quantile([0.1, 0.25, 0.5, .75, 0.9]).compute()

In [ ]:
(ddf.fare_amount + ddf.tip_amount).head()

In [ ]:
ddf.RatecodeID.isna().mean().compute()

### Dask is familiar

<img width="40%" src="https://docs.dask.org/en/latest/_images/dask-dataframe.svg"/>

We saw earlier that dask.dataframe mimics the pandas API. We could use the same keywords to get the same behavior. But perhaps more importantly, Dask feels familiar because it uses pandas to do dataframe operations. A `dask.dataframe.value_counts` is just a bunch of `pandas.value_counts` plus a bit of logic to combine the results. Dask Array is a bunch of NumPy arrays with some logic for how to work with them in parallel.

In [ ]:
import geopandas

zones = geopandas.read_file("./taxi_zones")
zones.head()

In [ ]:
zones.plot();

In [ ]:
center = zones.geometry.centroid.to_crs(crs="EPSG:4326")
zones["lng"] = center.x
zones["lat"] = center.y
# for memory savings
zones['borough'] = zones['borough'].astype('category')

In [ ]:
df[['PULocationID', 'DOLocationID']].head()

In [ ]:
zones[['LocationID', 'borough', 'lat', 'lng']].rename(
    columns=lambda x: f"DO{x}"
)

In [ ]:
df2 = pd.merge(
    df,
    zones[['LocationID', 'borough', 'lat', 'lng']].rename(
        columns=lambda x: f"DO{x}"
    )
)
df3 = pd.merge(
    df2,
    zones[['LocationID', 'borough', 'lat', 'lng']].rename(
        columns=lambda x: f"PU{x}"
    )
)
df3.head()

In [ ]:
import numpy as np


def gcd(lat1, lng1, lat2, lng2):
    '''
    Calculate great circle distance.
    http://www.johndcook.com/blog/python_longitude_latitude/

    Parameters
    ----------
    lat1, lng1, lat2, lng2: float or array of float

    Returns
    -------
    distance:
      distance from ``(lat1, lng1)`` to ``(lat2, lng2)`` in kilometers.
    '''
    # python2 users will have to use ascii identifiers
    ϕ1 = np.deg2rad(90 - lat1)
    ϕ2 = np.deg2rad(90 - lat2)

    θ1 = np.deg2rad(lng1)
    θ2 = np.deg2rad(lng2)

    cos = (np.sin(ϕ1) * np.sin(ϕ2) * np.cos(θ1 - θ2) +
           np.cos(ϕ1) * np.cos(ϕ2))
    arc = np.arccos(cos)
    return arc * 6373

In [ ]:
gcd(df3.PUlat, df3.PUlng, df3.DOlat, df3.DOlng)

In [ ]:
ddf2 = dd.merge(
    ddf,
    zones[['LocationID', 'borough', 'lat', 'lng']].rename(
        columns=lambda x: f"DO{x}"
    )
)
ddf3 = dd.merge(
    ddf2,
    zones[['LocationID', 'borough', 'lat', 'lng']].rename(
        columns=lambda x: f"PU{x}"
    )
)
ddf3.head()

In [ ]:
distance = gcd(ddf3.PUlat, ddf3.PUlng, ddf3.DOlat, ddf3.DOlng)
distance.head()

In [ ]:
distance.quantile([0.1, 0.25, 0.5, 0.75, .9]).compute()

## Parallelizing Custom Code

Not all problems fit in the big array / big dataframe model. There's often some bespoke munging that needs to be done before data can be loaded into an array or dataframe. `dask.delayed` helps here.

In [ ]:
from time import sleep

def inc(x):
    sleep(1)
    return x + 1

def add(x, y):
    sleep(1)
    return x + y

In [ ]:
%%time
# This takes three seconds to run because we call each
# function sequentially, one after the other

x = inc(1)
y = inc(2)
z = add(x, y)

In [ ]:
import dask

In [ ]:
%%time
# This runs immediately, all it does is build a graph

x = dask.delayed(inc)(1)
y = dask.delayed(inc)(2)
z = dask.delayed(add)(x, y)

In [ ]:
%%time
# This actually runs the computation in parallel
z.compute()

In [ ]:
z

In [ ]:
z.visualize()

In [ ]:
data = [1, 2, 3, 4, 5, 6, 7, 8]

In [ ]:
%%time
results = []
for x in data:
    y = inc(x)
    results.append(y)
    
total = sum(results)

In [ ]:
%%time
results = []
for x in data:
    y = dask.delayed(inc)(x)
    results.append(y)
    
total = sum(results)

In [ ]:
%time total.compute()

In [ ]:
total.visualize(rankdir="LR")

In [ ]:
client.restart()

### Privacy Example

`cape-privacy` is

> A Python library supporting data transformations and collaborative privacy policies

Dask support is on the roadmap, but doesn't currently exists. Let's use this as an example to see what porting dask code to python looks like.

https://github.com/capeprivacy/cape-python/blob/master/examples/tutorials/credit/mask_credit_data_in_pandas.ipynb

In [3]:
import matplotlib.pylab as plt
import pandas as pd

import cape_privacy as cape
from cape_privacy.pandas import dtypes
from cape_privacy.pandas import transformations as tfms

In [4]:
url = "https://raw.githubusercontent.com/capeprivacy/cape-python/master/examples/tutorials/credit/data/credit_with_pii.csv"
credit = pd.read_csv(url, parse_dates=["Application_date"])
credit.head()

,Name,City,Street_address,Salary,Application_date,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,Robert Fitzgerald,Greenmouth,349 Alexander Coves Apt. 799,36964.79,2018-08-03,67,male,2,own,NaN,little,1169,6,radio/TV,good
1,Daniel Kim,North Jessica,349 Jesse Park Suite 888,87884.39,2018-08-07,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,Frederick Jordan,Lake Erika,389 Graham Drive Suite 973,41157.73,2018-05-14,49,male,1,own,little,NaN,2096,12,education,good
3,Tara Rojas,East Jessica,01579 Ramirez Drives Apt. 587,36214.80,2018-04-30,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,Gail Donovan,Randyport,467 Christopher Well,41353.49,2018-05-27,53,male,2,free,little,little,4870,24,car,bad


In [56]:
tokenize_name = tfms.Tokenizer()
tokenize_sex = tfms.Tokenizer(max_token_len=10)
perturb_age = tfms.NumericPerturbation(dtype=dtypes.Integer, min=-5, max=5)
perturb_application_date = tfms.DatePerturbation(frequency="DAY", min=-3, max=3)
redact_location = tfms.ColumnRedact(columns=["Street_address", "City"])
round_salary = tfms.NumericRounding(dtype=dtypes.Float, precision=-3)

def mask(df):
    df = df.copy()
    name = tokenize_name(df['Name'])
    sex = tokenize_sex(df['Sex'])
    age = perturb_age(df['Age'])
    application_date = perturb_application_date(df['Application_date'])
    salary = round_salary(df['Salary'])
    df = redact_location(df)
    
    return df.assign(
        Name=name, Sex=sex, Age=age, Application_date=application_date,
        Salar=salary
    )

caped_df = mask(credit)
caped_df.head()

,Name,Salary,Application_date,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk,Salar
0,c7ca19babf7be43b63fbf0cd98c5ed0d20321f811829de...,36964.79,2018-07-31,68,98439e2dc2,2,own,NaN,little,1169,6,radio/TV,good,37000.0
1,1ff894ac2d0fd34b6d5d7a24867da33406858f0b87c93b...,87884.39,2018-08-06,21,e926a69cbb,2,own,little,moderate,5951,48,radio/TV,bad,88000.0
2,b0dc4a975f0118a960784582b5908481a16e98808e72fc...,41157.73,2018-05-13,49,98439e2dc2,1,own,little,NaN,2096,12,education,good,41000.0
3,f319b90d2ce720fe8035b15db87426bcd09874ef699019...,36214.80,2018-04-30,48,98439e2dc2,2,free,little,little,7882,42,furniture/equipment,good,36000.0
4,d5ff340dcb6e3d3f6e9c7a0d284c0ac0205a4d7b0d0cd3...,41353.49,2018-05-27,53,98439e2dc2,2,free,little,little,4870,24,car,bad,41000.0


In [57]:
tokenize_name(credit.Name)

0      c7ca19babf7be43b63fbf0cd98c5ed0d20321f811829de...
1      1ff894ac2d0fd34b6d5d7a24867da33406858f0b87c93b...
2      b0dc4a975f0118a960784582b5908481a16e98808e72fc...
3      f319b90d2ce720fe8035b15db87426bcd09874ef699019...
4      d5ff340dcb6e3d3f6e9c7a0d284c0ac0205a4d7b0d0cd3...
                             ...                        
995    a7564f309651e72bbf6661f84781d8f75a6ba74775f289...
996    c25a9f04b15095330f9acca6cf4d2cc71a8facf503c757...
997    9e9c6d3d9934d790abb4a5d0a4f383a111856a6987176b...
998    648f4294e2db1adf660debed0b78514a0dfede5ca926f1...
999    58a6bcafe8c6df370068e652fdf73979c82346a38d3d24...
Name: Name, Length: 1000, dtype: object

In [58]:
import dask.dataframe as dd

dcredit = dd.from_pandas(credit, 4)
dcredit

,Name,City,Street_address,Salary,Application_date,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
npartitions=4,,,,,,,,,,,,,,,
0,object,object,object,float64,datetime64[ns],int64,object,int64,object,object,object,int64,int64,object,object
250,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
750,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [59]:
dcredit.Name.map_partitions(tokenize_name)

Dask Series Structure:
npartitions=4
0      object
250       ...
500       ...
750       ...
999       ...
Name: Name, dtype: object
Dask Name: Tok, 12 tasks

In [75]:
dcape_df = dcredit.map_partitions(mask)
dcape_df

,Name,Salary,Application_date,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk,Salar
npartitions=4,,,,,,,,,,,,,,
0,object,float64,datetime64[ns],int64,object,int64,object,object,object,int64,int64,object,object,float32
250,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,...,...,...,...,...,...,...,...,...,...,...,...,...,...
750,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [76]:
dcape_df.head()

,Name,Salary,Application_date,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk,Salar
0,c7ca19babf7be43b63fbf0cd98c5ed0d20321f811829de...,36964.79,2018-08-05,70,98439e2dc2,2,own,NaN,little,1169,6,radio/TV,good,37000.0
1,1ff894ac2d0fd34b6d5d7a24867da33406858f0b87c93b...,87884.39,2018-08-09,26,e926a69cbb,2,own,little,moderate,5951,48,radio/TV,bad,88000.0
2,b0dc4a975f0118a960784582b5908481a16e98808e72fc...,41157.73,2018-05-12,49,98439e2dc2,1,own,little,NaN,2096,12,education,good,41000.0
3,f319b90d2ce720fe8035b15db87426bcd09874ef699019...,36214.80,2018-04-27,45,98439e2dc2,2,free,little,little,7882,42,furniture/equipment,good,36000.0
4,d5ff340dcb6e3d3f6e9c7a0d284c0ac0205a4d7b0d0cd3...,41353.49,2018-05-29,57,98439e2dc2,2,free,little,little,4870,24,car,bad,41000.0


In [77]:
dcape_df.compute()

KeyError: 0

## Scalable Machine Learning with Dask-ML



In [82]:
import pandas as pd
from cape_privacy.pandas import transformations as tfms

perturb_application_date = tfms.DatePerturbation(frequency="DAY", min=-3, max=3)
s = pd.Series(pd.date_range('2000', periods=12), index=list(range(1, 13)))
s

1    2000-01-01
2    2000-01-02
3    2000-01-03
4    2000-01-04
5    2000-01-05
6    2000-01-06
7    2000-01-07
8    2000-01-08
9    2000-01-09
10   2000-01-10
11   2000-01-11
12   2000-01-12
dtype: datetime64[ns]

In [83]:
perturb_application_date(s)

KeyError: 0

In [ ]:
kjk